# Xpedia 

Your goal of this competition is to predict the booking outcome (hotel cluster) for a user event, based on their search and other attributes associated with that user event. Expedia is interested in predicting which hotel group a user is going to book. Expedia has in-house algorithms to form hotel clusters, where similar hotels for a search (based on historical price, customer star ratings, geographical locations relative to city center, etc) are grouped together.

### MVP3:
 - Recommender
 - Investigate how to use a recommender system for this problem
  * collaborative filtering (on user or item level)
  * hybrid model combining use clicks/booking data & search features


https://www.kaggle.com/c/expedia-hotel-recommendations/data

In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from collections import Counter
import seaborn as sns

pd.set_option('display.max_columns', None) 
pd.set_option('display.float_format', lambda x: '%.5f' % x)

## Load data

In [3]:
data_dir = './expedia-hotel-recommendations/'

destinations = pd.read_csv(data_dir + 'destinations.csv')
train = pd.read_csv(data_dir + 'train.csv')
train_bookings = train[train['is_booking'] == 1]
test = pd.read_csv(data_dir + 'test.csv')
sample_submission = pd.read_csv(data_dir + 'sample_submission.csv')

In [4]:
train_sample = train.sample(500000)
train_sample.to_csv(data_dir + './train_sample.csv', index=False)

## (Pure) collaborative filtering

https://realpython.com/build-recommendation-engine-collaborative-filtering/

To build a system that can automatically recommend items to users based on the preferences of other users, the first step is to find similar users or items. The second step is to predict the ratings of the items that are not yet rated by a user. So, you will need the answers to these questions:

How do you determine which users or items are similar to one another?
Given that you know which users are similar, how do you determine the rating that a user would give to an item based on the ratings of similar users?
How do you measure the accuracy of the ratings you calculate?

* https://towardsdatascience.com/prototyping-a-recommender-system-step-by-step-part-1-knn-item-based-collaborative-filtering-637969614ea

* https://towardsdatascience.com/prototyping-a-recommender-system-step-by-step-part-2-alternating-least-square-als-matrix-4a76c58714a1

### Train-test split

In [5]:
import datetime as dt

test_threshold = "2014-10-01"

# Start only with bookings only : note, if we want to include clicks, we need to filter the clicks from the
# validation split
train_set = train_bookings[train_bookings['date_time'] < test_threshold]
test_set = train_bookings[train_bookings['date_time'] >= test_threshold]

In [13]:
def create_feedback_matrix(df, n_clusters=100):
    
    user_bookings = (df[['user_id', 'hotel_cluster']]
                     .groupby('user_id')
                     .agg({'hotel_cluster': list})
                     .reset_index())
    
    n_users = len(set(df['user_id']))
    
    # Fill feedback matrix
    feedback = np.zeros([n_users, n_clusters])
    for user in user_bookings.index:
        feedback[user, user_bookings.loc[user, 'hotel_cluster']] = 1
        
    return feedback

def create_y(df): 
    y_tmp = df.groupby('user_id').agg({'hotel_cluster': list})['hotel_cluster'].values

    return [
        y_i for y_i in y_tmp
    ]

In [7]:
X_train = create_feedback_matrix(train_set)

y_train = create_y(train_set)
X_train.shape

(704207, 100)

In [14]:
X_test = create_feedback_matrix(test_set)
y_test = create_y(test_set)

X_test.shape

(306682, 100)

In [15]:
from sklearn.base import BaseEstimator
from sklearn.decomposition import NMF
from ml_metrics import mapk


class ALS(BaseEstimator):
    
    def __init__(self, n_components=2, cutoff=5):
        self.n_components = n_components
        self.cutoff = cutoff

    def fit(self, X, y=None):
        
        self._model = self._get_model()
        
        return self._model.fit(X)
    
    def transform(self, X, y=None):
        return self.model.transform(X)
    
    def predict(self, X):

        W = self._model.fit_transform(X)
        H = self._model.components_
        V = np.dot(W, H)
        
        return np.argpartition(V, -self.cutoff)[:, -self.cutoff:]
    
    def score(self, X, y):
        """ TODO creates a prediction per user instead of per search query. """
        predictions = self.predict(X)
        return mapk(actual=y, predicted=predictions, k=self.cutoff)
    
    def _get_model(self):
        return NMF(
            n_components=self.n_components, 
            init='random', random_state=0, max_iter=300)


In [17]:
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV

param_search = {
    'n_components' : [5, 10]
}

tscv = TimeSeriesSplit(n_splits=2)
gsearch = GridSearchCV(estimator=ALS(), cv=tscv,
                       param_grid=param_search)

gsearch.fit(X_train, y_train)

pd.DataFrame(gsearch.cv_results_).sort_values('rank_test_score')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_components,params,split0_test_score,split1_test_score,mean_test_score,std_test_score,rank_test_score
1,9.17724,0.11843,9.57038,2.83471,10,{'n_components': 10},0.17007,0.17168,0.17087,0.00081,1
0,8.09748,1.78384,8.65820,1.09230,5,{'n_components': 5},0.13781,0.13739,0.13760,0.00021,2


In [18]:
estimator = gsearch.best_estimator_

In [20]:
print("Test MAP at K: ")

estimator.score(X=X_test, y=y_test)

Test MAP at K: 


0.15138185934906156

## Archive

### Evaluation

https://github.com/benhamner/Metrics/blob/master/Python/ml_metrics/average_precision.py

Is it correct that 'trailing' wrong predictions do not harm the average precision?


In [33]:
from ml_metrics import apk, mapk
print("Train Mean Average Precision at K: ")

actual = train_set['hotel_cluster'].values.reshape(-1, 1)

mapk(actual=actual, predicted=predictions, k=cutoff)

Train Mean Average Precision at K: 


NameError: name 'predictions' is not defined

In [82]:
print("Validation Mean Average Precisioon at K: ")

actual = validation_set['hotel_cluster'].values.reshape(-1, 1)

mapk(actual=actual, predicted=predictions, k=cutoff)

Validation Mean Average Precisioon at K: 


0.044222090713400726

In [72]:
apk(actual=[5], predicted=[2, 5])

0.5

In [73]:
apk(actual=[5], predicted=[5, 2])

1.0

In [29]:
from collections import Counter

Counter(np.argmax(V, axis=1))

Counter({95: 159973, 48: 170061, 59: 148190, 42: 48789, 91: 50895})

In [295]:
from sklearn.decomposition import non_negative_factorization

W, H, n_iter = non_negative_factorization(feedback, n_components=2, solver='cd', init='random', random_state=0)

### ALS Explore

https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.non_negative_factorization.html

https://medium.com/logicai/non-negative-matrix-factorization-for-recommendation-systems-985ca8d5c16c

Can we use the mechanism to prepare food recommendations for people? Yes, and it’s easier than you may think. By multiplying W and H, we obtain initial V matrix approximation:

In [20]:
from sklearn.decomposition import NMF

als_model = NMF(n_components=2, init='random', random_state=0)

W = als_model.fit_transform(train_feedback)
H = als_model.components_

V = np.dot(W, H)
V.shape

cutoff = 5

predictions = np.argpartition(V, -cutoff)[:, -cutoff:]

In [77]:
print(W.shape) 
print(H.shape)

(577908, 5)
(5, 100)


https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.TimeSeriesSplit.html

In [17]:
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV

X = train_feedback

tscv = TimeSeriesSplit()

for train_index, test_index in tscv.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]


TRAIN: [    0     1     2 ... 96315 96316 96317] TEST: [ 96318  96319  96320 ... 192633 192634 192635]
TRAIN: [     0      1      2 ... 192633 192634 192635] TEST: [192636 192637 192638 ... 288951 288952 288953]
TRAIN: [     0      1      2 ... 288951 288952 288953] TEST: [288954 288955 288956 ... 385269 385270 385271]
TRAIN: [     0      1      2 ... 385269 385270 385271] TEST: [385272 385273 385274 ... 481587 481588 481589]
TRAIN: [     0      1      2 ... 481587 481588 481589] TEST: [481590 481591 481592 ... 577905 577906 577907]
